In [37]:
import pandas as pd
import numpy as np
import json

In [38]:
# import packages
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,KFold
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

from sklearn.tree import export_graphviz
import pydot
from IPython.display import Image

from matplotlib import pyplot

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model   
from sklearn import metrics 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import numpy as np
import os

#To ignore the convergence warnings
from  warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

from sklearn.model_selection import train_test_split
import math

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import time
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve

In [117]:
chunk_size = 5000000
index=0
df = pd.read_csv('C:/Users/santi/Downloads/ProjectTrainingData.csv', nrows=chunk_size, skiprows=index*chunk_size)

In [169]:
##Please note that all parameter tunning were done with a subset of train data. Then we used 5000000 rows to train and
#predict on test data. More details will be explained on the report

# def do_sampling(dataframe):
    sample_rate=0.01
    if sample_rate < 1.0:
        np.random.seed(999)
        r1 = np.random.uniform(0, 1, dataframe.shape[0])
        dataframe = dataframe.iloc[r1 < sample_rate,:]
        print ("testing with small sample of training data, ", dataframe.shape)
        return dataframe

# sampling to save time for parameter tuning and dismiss this block for the last time you run (to see final results)
df = do_sampling(df)

In [118]:
df.columns = ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']

df.describe()

,id,click,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06
mean,9.286742e+18,1.727814e-01,1.410214e+07,1.004997e+03,2.569298e-01,1.025367e+00,2.198924e-01,1.850876e+04,3.196484e+02,5.903483e+01,2.080636e+03,1.647253e+00,1.928114e+02,4.880397e+04,8.018017e+01
std,5.317660e+18,3.780582e-01,4.238302e+01,1.084606e+00,4.720870e-01,5.297618e-01,6.829048e-01,4.694439e+03,2.838763e+01,4.616373e+01,5.724374e+02,1.363017e+00,3.172836e+02,5.003945e+04,6.341580e+01
min,7.425597e+12,0.000000e+00,1.410210e+07,1.001000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.750000e+02,1.200000e+02,2.000000e+01,1.120000e+02,0.000000e+00,3.300000e+01,-1.000000e+00,1.300000e+01
25%,4.674182e+18,0.000000e+00,1.410211e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,1.716300e+04,3.200000e+02,5.000000e+01,1.872000e+03,0.000000e+00,3.500000e+01,-1.000000e+00,3.200000e+01
50%,9.389530e+18,0.000000e+00,1.410211e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,1.998500e+04,3.200000e+02,5.000000e+01,2.260000e+03,2.000000e+00,3.900000e+01,-1.000000e+00,6.100000e+01
75%,1.388679e+19,0.000000e+00,1.410220e+07,1.005000e+03,1.000000e+00,1.000000e+00,0.000000e+00,2.168300e+04,3.200000e+02,5.000000e+01,2.496000e+03,3.000000e+00,1.690000e+02,1.000840e+05,1.110000e+02
max,1.844674e+19,1.000000e+00,1.410221e+07,1.012000e+03,7.000000e+00,5.000000e+00,5.000000e+00,2.187400e+04,1.024000e+03,1.024000e+03,2.526000e+03,3.000000e+00,1.835000e+03,1.002480e+05,2.210000e+02


In [119]:
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,...,1,0,18993,320,50,2161,0,35,-1,157


In [120]:
# check unique values in each column
for col in df.columns:
    print(col, len(df[col].value_counts()))

id 5000000
click 2
hour 33
C1 7
banner_pos 7
site_id 3075
site_domain 3769
site_category 22
app_id 4545
app_domain 314
app_category 31
device_id 450552
device_ip 1310686
device_model 6335
device_type 4
device_conn_type 4
C14 828
C15 8
C16 9
C17 201
C18 4
C19 44
C20 168
C21 38


In [121]:
# check feature distribution
print(df.groupby('site_id')['click'].count().sort_values(ascending=False).head(10))
print(df.groupby('app_id')['click'].count().sort_values(ascending=False).head(10))

site_id
85f751fd    1842788
1fbe01fe     893609
e151e245     236752
d9750ee7     164452
5b08c53b     104274
856e6d3f      94672
5b4d2eda      82781
5ee41ff2      52472
6399eda6      46929
6256f5b4      38854
Name: click, dtype: int64
app_id
ecad2386    3157212
e2fcccd2     151265
7358e05e     109727
febd1138      95658
66f5e02e      83675
9c13b419      68286
54c5d545      56242
a5184c22      54177
f0d41ff1      42766
03528b27      37642
Name: click, dtype: int64


In [122]:
# check out the proportion
print(f"top1 site id: {df[df['site_id'] == '85f751fd'].shape[0] / df.shape[0]}")
print(f"top1 app id: {df[df['app_id'] == 'ecad2386'].shape[0] / df.shape[0]}")

top1 site id: 0.3685576
top1 app id: 0.6314424


In [123]:
print(f" top1 site id count + top1 app id count: {df[df['site_id'] == '1fbe01fe'].shape[0] + df[df['app_id'] == 'ecad2386'].shape[0]}")
print(f" total dataset count: {df.shape[0]}")

# each data is either "site id == 1fbe01fe" or "app_id ==  ecad2386"

 top1 site id count + top1 app id count: 4050821
 total dataset count: 5000000


In [124]:


def feature_engineering(dataframe):
    
    dataframe = dataframe.drop(['id'], axis=1)
    
    # extract week & hour
    dataframe['Date'] = pd.to_datetime(dataframe['hour'], format='%y%m%d%H')
    dataframe['Day'] = dataframe['Date'].dt.day
    dataframe['Weekday'] = dataframe['Date'].dt.dayofweek
    dataframe['Hour'] = dataframe['Date'].dt.hour
    print(dataframe.head())
    dataframe = dataframe.drop(['Date', 'hour'], axis=1)

    # extract new feature from app id and site id
    dataframe.loc[:, 'is_ecad2386'] = 0
    dataframe.loc[dataframe.app_id=='ecad2386', 'is_ecad2386'] = 1
    print(dataframe.head())
    
    return dataframe
    

df2 = feature_engineering(df)

for col in df2.columns:
    print(col, len(df2[col].value_counts()))

   click      hour    C1  banner_pos   site_id site_domain site_category  \
0      0  14102100  1005           0  1fbe01fe    f3845767      28905ebd   
1      0  14102100  1005           0  1fbe01fe    f3845767      28905ebd   
2      0  14102100  1005           0  1fbe01fe    f3845767      28905ebd   
3      0  14102100  1005           0  1fbe01fe    f3845767      28905ebd   
4      0  14102100  1005           1  fe8cc448    9166c161      0569f928   

     app_id app_domain app_category  ... C16   C17 C18  C19     C20  C21  \
0  ecad2386   7801e8d9     07d7df22  ...  50  1722   0   35      -1   79   
1  ecad2386   7801e8d9     07d7df22  ...  50  1722   0   35  100084   79   
2  ecad2386   7801e8d9     07d7df22  ...  50  1722   0   35  100084   79   
3  ecad2386   7801e8d9     07d7df22  ...  50  1722   0   35  100084   79   
4  ecad2386   7801e8d9     07d7df22  ...  50  2161   0   35      -1  157   

        Date  Day  Weekday  Hour  
0 2014-10-21   21        1     0  
1 2014-10-21   2

In [125]:
from sklearn import preprocessing

def feature_encoding(dataframe):
    feats_to_encode = ['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category',
                       'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model',
                      'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'Day', 'Weekday', 'Hour', 'Weekday']
    for column in feats_to_encode:
        enc = preprocessing.LabelEncoder()
        dataframe[column] = enc.fit_transform(dataframe[column])
    
    print(dataframe.info())
    print(dataframe.head())
    
    return dataframe

df3 = feature_encoding(df2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000000 entries, 0 to 4999999
Data columns (total 26 columns):
 #   Column            Dtype
---  ------            -----
 0   click             int64
 1   C1                int64
 2   banner_pos        int64
 3   site_id           int32
 4   site_domain       int32
 5   site_category     int32
 6   app_id            int32
 7   app_domain        int32
 8   app_category      int32
 9   device_id         int32
 10  device_ip         int32
 11  device_model      int32
 12  device_type       int64
 13  device_conn_type  int64
 14  C14               int64
 15  C15               int64
 16  C16               int64
 17  C17               int64
 18  C18               int64
 19  C19               int64
 20  C20               int64
 21  C21               int64
 22  Day               int64
 23  Weekday           int64
 24  Hour              int64
 25  is_ecad2386       int64
dtypes: int32(9), int64(17)
memory usage: 820.2 MB
None
   click  C1  banne

In [126]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing


def standardize_data(dataframe):
    scaler = preprocessing.StandardScaler().fit(dataframe)
    dataframe_scaled = scaler.transform(dataframe)
    return pd.DataFrame(dataframe_scaled, columns=dataframe.columns)

def split_data(dataframe):
    x_feats = dataframe.columns[dataframe.columns != 'click']
    X = dataframe[x_feats]
    y = dataframe['click']

    print(X.head())
    print(y.head())
    
    X = standardize_data(X)
    return X, y


X, y = split_data(df3)

   C1  banner_pos  site_id  site_domain  site_category  app_id  app_domain  \
0   2           0      392         3571              2    4188         141   
1   2           0      392         3571              2    4188         141   
2   2           0      392         3571              2    4188         141   
3   2           0      392         3571              2    4188         141   
4   2           1     3043         2164              0    4188         141   

   app_category  device_id  device_ip  ...  C16  C17  C18  C19  C20  C21  Day  \
0             0     298028    1135371  ...    2   46    0    2    0   19    0   
1             0     298028     771212  ...    2   46    0    2   63   19    0   
2             0     298028     920581  ...    2   46    0    2   63   19    0   
3             0     298028    1188570  ...    2   46    0    2   63   19    0   
4             0     298028     770002  ...    2   90    0    2    0   34    0   

   Weekday  Hour  is_ecad2386  
0        0  

In [127]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [170]:
#Now that we have preprocess, clean, and encode the data we can start working on XGBOOST
#XGBoost is great but it has so many parameters that is difficult to train. 
#We will try different approaches to try and create the best model

# XGBOOST

In [128]:
import xgboost as xgb
from xgboost import XGBClassifier

In [129]:
data_dmatrix = xgb.DMatrix(data=X ,label=y)

In [46]:
xgbc = XGBClassifier()
print(xgbc)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=15, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=42, silent=None,
       subsample=1, verbosity=1) 

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_delta_step=0,
              max_depth=15, min_child_weight=1, missing=None,
              monotone_constraints=None, n_estimators=100, n_jobs=1,
              nthread=None, num_parallel_tree=None, objective='multi:softprob',
              predictor=None, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, seed=42, silent=None, subsample=1,
              tree_method=None, ...)

In [47]:
xgbc.fit(x_train, y_train)

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:14:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [49]:
from sklearn.metrics import log_loss

y_pred = xgbc.predict_proba(x_test)

log_loss = sklearn.metrics.log_loss(y_test, y_pred, eps=1e-15, normalize=True, sample_weight=None, labels=None)

log_loss

0.4192677120835939

In [115]:
scores = cross_val_score(xgbc, X, y, cv=5, scoring = 'neg_log_loss') # cross-validation scores

print("Cross validation log loss: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)) #estimate mean and variance from cross validation

scores

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:30:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:30:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:30:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:30:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross validation log loss: -0.65 (+/- 0.41)


array([-0.42653192, -0.47475223, -0.97307421, -0.80279869, -0.5736258 ])

# Hyperparameter Tunning

In [ ]:
#Here we are using pipeline and randomize search to get the best parameters
#It takes a long time to run and we originally had more parameters to tune

In [174]:
steps = [('scaler', StandardScaler()), ('XGBoost', xgb.XGBClassifier(eval_metric = 'logloss', random_state = 42))] 

In [175]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps)

In [176]:
parameters = {'XGBoost__booster': ['gbtree', 'gblinear', 'dart'],
              'XGBoost__max_depth': np.arange(3, 18, 1),
              'XGBoost__reg_alpha': [1e-5, 1e-2, 0.1, 1],
              'XGBoost__learning_rate': np.arange(0.1,1,.2),
              'XGBoost__eval_metric' : ['logloss'],
              'XGBoost__colsample_bytree': np.arange(0.1, 1.05, 0.05),
              'XGBoost__objective': ['reg:logistic','binary:logistic']}

In [177]:
from sklearn.model_selection import RandomizedSearchCV
model = RandomizedSearchCV(pipeline, parameters, n_iter=5, scoring= "neg_log_loss", cv=4)

In [178]:
start_time = time.time()

In [179]:
model.fit(x_train, y_train)

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

RandomizedSearchCV(cv=4,
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('XGBoost',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            enable_categorical=False,
                                                            eval_metric='logloss',
                                                            gamma=None,
                                                            gpu_id=None,
                                                            importance_type=None,
                                              

In [180]:
print('Tuned Model Parameters: {}'.format(model.best_params_))

Tuned Model Parameters: {'XGBoost__reg_alpha': 0.1, 'XGBoost__objective': 'reg:logistic', 'XGBoost__max_depth': 12, 'XGBoost__learning_rate': 0.1, 'XGBoost__eval_metric': 'logloss', 'XGBoost__colsample_bytree': 0.3500000000000001, 'XGBoost__booster': 'dart'}


In [181]:
elapsed_time = (time.time() - start_time)/60
print('Time to tune the model: {0:0.2f} min.'.format(elapsed_time))

Time to tune the model: 153.58 min.


In [116]:
scores = cross_val_score(model, X, y, cv=5, scoring = 'neg_log_loss') # cross-validation scores

print("Cross validation log loss: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)) #estimate mean and variance from cross validation

scores

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

[13:31:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[13:31:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:31:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[13:37:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[13:37:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:37:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[13:37:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "colsample_bytree", "max_depth" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of 

KeyboardInterrupt: 

In [182]:
import sklearn
from sklearn.metrics import log_loss

y_pred = model.predict_proba(x_test)

log_loss = sklearn.metrics.log_loss(y_test, y_pred, eps=1e-15, normalize=True, sample_weight=None, labels=None)



In [183]:
log_loss

0.39376112972468896

# Tunning with Gridsearch CV

In [171]:
#Similar, but now using gridearch to tune each paramete individually

In [130]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [131]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
xgb1.fit(x_train, y_train)

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:40:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [85]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='neg_log_loss',n_jobs=4, cv=5)
gsearch1.fit(x_train,y_train)
gsearch1.best_params_, gsearch1.best_score_

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:01:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'max_depth': 5, 'min_child_weight': 5}, -0.41215822752764647)

In [87]:
param_test2 = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate=0.1, n_estimators=140, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2, scoring='neg_log_loss',n_jobs=4, cv=5)
gsearch2.fit(x_train,y_train)
gsearch2.best_params_, gsearch2.best_score_

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:03:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'max_depth': 5, 'min_child_weight': 5}, -0.41215822752764647)

In [88]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=5, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='neg_log_loss',n_jobs=4, cv=5)
gsearch3.fit(x_train,y_train)
gsearch3.best_params_, gsearch3.best_score_

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:08:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'gamma': 0.4}, -0.41205602469977387)

In [89]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=5,
 min_child_weight=5, gamma=.4, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='neg_log_loss',n_jobs=4,cv=5)
gsearch4.fit(x_train,y_train)
gsearch4.best_params_, gsearch4.best_score_

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:12:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'colsample_bytree': 0.6, 'subsample': 0.9}, -0.4114643533372038)

In [91]:
param_test5 = {
 'subsample':[i/100.0 for i in range(75,90,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=5,
 min_child_weight=5, gamma=.4, subsample=0.9, colsample_bytree=0.6,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test5, scoring='neg_log_loss',n_jobs=4, cv=5)
gsearch5.fit(x_train,y_train)
gsearch5.best_params_, gsearch5.best_score_

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:15:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'colsample_bytree': 0.75, 'subsample': 0.85}, -0.4118991290758718)

In [92]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=177, max_depth=5,
 min_child_weight=5, gamma=0.4, subsample=0.85, colsample_bytree=0.75,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='neg_log_loss',n_jobs=4, cv=5)
gsearch6.fit(x_train,y_train)
gsearch6.best_params_, gsearch6.best_score_

C:\Users\santi\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:16:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


({'reg_alpha': 1e-05}, -0.41189912643461674)

In [132]:
xgb2 = XGBClassifier(learning_rate =0.1, n_estimators=177, max_depth=5,
 min_child_weight=5, gamma=0.4, subsample=0.85, colsample_bytree=0.75,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27)

xgb2.fit(x_train, y_train)


[13:54:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.75,
              enable_categorical=False, gamma=0.4, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=177, n_jobs=4, nthread=4, num_parallel_tree=1,
              predictor='auto', random_state=27, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, seed=27, subsample=0.85, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [133]:
import sklearn
from sklearn.metrics import log_loss

y_pred = xgb2.predict_proba(x_test)

log_loss = sklearn.metrics.log_loss(y_test, y_pred, eps=1e-15, normalize=True, sample_weight=None, labels=None)

log_loss

0.4052140240763463

In [173]:
#Best model as a team. We will use this one to make predictions. We will fit all 50000000 million rows on it and predict on test. 

# Making Predictions on Test

In [134]:
Test = pd.read_csv('C:/Users/santi/Downloads/ProjectTestData.csv', sep=',')

In [161]:
Test_org = pd.read_csv('C:/Users/santi/Downloads/ProjectTestData.csv', sep=',')

In [135]:
Test.columns = ['id', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']

Test.describe()

,id,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07
mean,9.223718e+18,1.410288e+07,1.004967e+03,2.462825e-01,1.006340e+00,3.536481e-01,2.048630e+04,3.194561e+02,5.976233e+01,2.316581e+03,1.388272e+00,2.253101e+02,5.226684e+04,9.335834e+01
std,5.326120e+18,3.051849e+02,9.829727e-01,4.845275e-01,4.582598e-01,8.665138e-01,4.530735e+03,2.350921e+01,4.634691e+01,5.653348e+02,1.327725e+00,3.270714e+02,5.000977e+04,8.186552e+01
min,1.398959e+12,1.410210e+07,1.001000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.750000e+02,1.200000e+02,2.000000e+01,1.120000e+02,0.000000e+00,3.300000e+01,-1.000000e+00,1.000000e+00
25%,4.610782e+18,1.410281e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,1.925100e+04,3.200000e+02,5.000000e+01,2.104000e+03,0.000000e+00,3.500000e+01,-1.000000e+00,2.300000e+01
50%,9.225027e+18,1.410301e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.222100e+04,3.200000e+02,5.000000e+01,2.545000e+03,2.000000e+00,3.900000e+01,1.000280e+05,5.200000e+01
75%,1.383742e+19,1.410311e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.356000e+04,3.200000e+02,5.000000e+01,2.681000e+03,3.000000e+00,2.970000e+02,1.001110e+05,2.120000e+02
max,1.844674e+19,1.410312e+07,1.012000e+03,7.000000e+00,5.000000e+00,5.000000e+00,2.434900e+04,1.024000e+03,1.024000e+03,2.793000e+03,3.000000e+00,1.839000e+03,1.002480e+05,2.550000e+02


In [162]:
Test_org.columns = ['id', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']

Test_org.describe()

,id,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
count,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07,1.301534e+07
mean,9.223718e+18,1.410288e+07,1.004967e+03,2.462825e-01,1.006340e+00,3.536481e-01,2.048630e+04,3.194561e+02,5.976233e+01,2.316581e+03,1.388272e+00,2.253101e+02,5.226684e+04,9.335834e+01
std,5.326120e+18,3.051849e+02,9.829727e-01,4.845275e-01,4.582598e-01,8.665138e-01,4.530735e+03,2.350921e+01,4.634691e+01,5.653348e+02,1.327725e+00,3.270714e+02,5.000977e+04,8.186552e+01
min,1.398959e+12,1.410210e+07,1.001000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.750000e+02,1.200000e+02,2.000000e+01,1.120000e+02,0.000000e+00,3.300000e+01,-1.000000e+00,1.000000e+00
25%,4.610782e+18,1.410281e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,1.925100e+04,3.200000e+02,5.000000e+01,2.104000e+03,0.000000e+00,3.500000e+01,-1.000000e+00,2.300000e+01
50%,9.225027e+18,1.410301e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.222100e+04,3.200000e+02,5.000000e+01,2.545000e+03,2.000000e+00,3.900000e+01,1.000280e+05,5.200000e+01
75%,1.383742e+19,1.410311e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.356000e+04,3.200000e+02,5.000000e+01,2.681000e+03,3.000000e+00,2.970000e+02,1.001110e+05,2.120000e+02
max,1.844674e+19,1.410312e+07,1.012000e+03,7.000000e+00,5.000000e+00,5.000000e+00,2.434900e+04,1.024000e+03,1.024000e+03,2.793000e+03,3.000000e+00,1.839000e+03,1.002480e+05,2.550000e+02


In [136]:
def feature_engineering(dataframe):
    
    dataframe = dataframe.drop(['id'], axis=1)
    
    # extract week & hour
    dataframe['Date'] = pd.to_datetime(dataframe['hour'], format='%y%m%d%H')
    dataframe['Day'] = dataframe['Date'].dt.day
    dataframe['Weekday'] = dataframe['Date'].dt.dayofweek
    dataframe['Hour'] = dataframe['Date'].dt.hour
    print(dataframe.head())
    dataframe = dataframe.drop(['Date', 'hour'], axis=1)

    # extract new feature from app id and site id
    dataframe.loc[:, 'is_ecad2386'] = 0
    dataframe.loc[dataframe.app_id=='ecad2386', 'is_ecad2386'] = 1
    print(dataframe.head())
    
    return dataframe
    

Test2 = feature_engineering(Test)

for col in Test2.columns:
    print(col, len(Test2[col].value_counts()))

       hour    C1  banner_pos   site_id site_domain site_category    app_id  \
0  14102809  1005           1  85f751fd    c4e18dd6      50e219e0  1dc72b4d   
1  14103120  1007           0  85f751fd    c4e18dd6      50e219e0  8311368f   
2  14103109  1005           0  4e7614cf    c1aa3c04      f028772b  ecad2386   
3  14103101  1002           0  48c42b43    de0f0f82      50e219e0  ecad2386   
4  14103119  1005           1  e151e245    7e091613      f028772b  ecad2386   

  app_domain app_category device_id  ... C16   C17  C18   C19     C20  C21  \
0   2347f47a     0f2161f8  cd915ca3  ...  50   761    3   175  100075   23   
1   1dc9b529     0f2161f8  ec0aff16  ...  50  2788    3   295  100194  240   
2   7801e8d9     07d7df22  a99f214a  ...  50  2776    0    35      -1   79   
3   7801e8d9     07d7df22  dfb9b781  ...  50  2684    2  1327      -1   52   
4   7801e8d9     07d7df22  a99f214a  ...  50  2761    2    35  100148   13   

                 Date  Day  Weekday  Hour  
0 2014-10-28

In [137]:
from sklearn import preprocessing

def feature_encoding(dataframe):
    feats_to_encode = ['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category',
                       'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model',
                      'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'Day', 'Weekday', 'Hour', 'Weekday']
    for column in feats_to_encode:
        enc = preprocessing.LabelEncoder()
        dataframe[column] = enc.fit_transform(dataframe[column])
    
    print(dataframe.info())
    print(dataframe.head())
    
    return dataframe

Test3 = feature_encoding(Test2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13015341 entries, 0 to 13015340
Data columns (total 25 columns):
 #   Column            Dtype
---  ------            -----
 0   C1                int64
 1   banner_pos        int64
 2   site_id           int32
 3   site_domain       int32
 4   site_category     int32
 5   app_id            int32
 6   app_domain        int32
 7   app_category      int32
 8   device_id         int32
 9   device_ip         int32
 10  device_model      int32
 11  device_type       int64
 12  device_conn_type  int64
 13  C14               int64
 14  C15               int64
 15  C16               int64
 16  C17               int64
 17  C18               int64
 18  C19               int64
 19  C20               int64
 20  C21               int64
 21  Day               int64
 22  Weekday           int64
 23  Hour              int64
 24  is_ecad2386       int64
dtypes: int32(9), int64(16)
memory usage: 2.0 GB
None
   C1  banner_pos  site_id  site_domain  site_ca

In [150]:
def standardize_data(dataframe):
    scaler = preprocessing.StandardScaler().fit(dataframe)
    dataframe_scaled = scaler.transform(dataframe)
    return pd.DataFrame(dataframe_scaled, columns=dataframe.columns)
    
Testx = standardize_data(Test3)
  


In [151]:
Testx

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,...,C16,C17,C18,C19,C20,C21,Day,Weekday,Hour,is_ecad2386
0,-0.032766,1.603125,0.258204,0.336624,-0.577628,-2.955802,-1.579632,-0.087309,1.290011,0.041583,...,-0.20333,-1.549806,1.213902,0.063167,0.207551,-0.881722,-0.233453,-1.835331,-0.401533,-1.236863
1,2.005255,-0.520880,0.258204,0.336624,-0.577628,-1.211893,-1.721939,-0.087309,2.215258,-0.227975,...,-0.20333,1.533030,1.213902,0.320858,1.982810,1.649849,0.750316,0.658865,1.471152,-1.236863
2,-0.032766,-0.520880,-0.576807,0.283218,1.390429,0.556853,0.313049,-0.543353,0.199827,0.831661,...,-0.20333,1.450454,-1.045602,-0.709904,-0.884916,-0.087504,0.750316,0.658865,-0.401533,0.808497
3,-2.070787,-0.520880,-0.668678,0.769717,-0.577628,0.556853,0.313049,-0.543353,1.842881,-0.479604,...,-0.20333,0.906829,0.460734,2.962184,-0.884916,-0.434974,0.750316,0.658865,-1.763485,0.808497
4,-0.032766,1.603125,1.674049,-0.851667,1.390429,0.556853,0.313049,-0.543353,0.199827,-1.554918,...,-0.20333,1.360997,0.460734,-0.709904,1.241493,-1.129916,0.750316,0.658865,1.300908,0.808497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13015336,-0.032766,-0.520880,-1.243387,-1.990724,1.506197,0.556853,0.313049,-0.543353,0.199827,1.413355,...,-0.20333,1.202726,1.213902,-0.838749,1.573135,1.699488,0.422393,-0.172534,-0.231289,0.808497
13015337,-0.032766,1.603125,1.674049,-0.851667,1.390429,0.556853,0.313049,-0.543353,0.199827,-0.411979,...,-0.20333,1.085744,1.213902,-0.323369,-0.884916,-0.881722,0.422393,-0.172534,-0.571777,0.808497
13015338,-0.032766,-0.520880,-1.502669,1.116858,1.390429,0.556853,0.313049,-0.543353,0.199827,0.436209,...,-0.20333,0.060425,-1.045602,0.900661,-0.884916,1.550572,0.750316,0.658865,1.981884,0.808497
13015339,-0.032766,-0.520880,0.258204,0.336624,-0.577628,0.026098,1.508427,2.306923,0.199827,-1.244088,...,-0.20333,1.409166,-0.292434,-0.130101,1.943794,-0.186781,0.750316,0.658865,-0.401533,-1.236863


In [152]:
x_test

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,...,C16,C17,C18,C19,C20,C21,Day,Weekday,Hour,is_ecad2386
3577888,-0.064673,-0.549907,-0.419588,-0.881008,-0.711134,0.522749,0.149793,-0.460784,0.216689,1.556229,...,4.107687,-0.604578,0.258799,-0.523571,0.478766,-0.839825,-0.610195,-0.610195,2.237796,0.763987
4993932,-0.064673,-0.549907,0.236414,0.391922,-0.433329,-1.534643,0.007716,-0.084049,0.216689,-0.616764,...,-0.175430,1.224535,-1.208534,-0.621263,-0.824774,1.478396,1.638819,1.638819,-0.113109,-1.308923
4094900,-0.064673,-0.549907,-0.295839,-2.128501,1.511309,0.522749,0.149793,-0.460784,0.216689,0.531972,...,-0.175430,0.359863,0.992465,-0.523571,-0.824774,-1.011546,1.638819,1.638819,-0.784796,0.763987
4420497,-0.064673,-0.549907,0.236414,0.391922,-0.433329,-0.971586,-1.839277,2.678678,0.300317,-0.142834,...,-0.175430,1.324305,0.992465,-0.621263,-0.824774,-0.152945,1.638819,1.638819,-0.616875,-1.308923
634465,-0.064673,-0.549907,0.236414,0.391922,-0.433329,-2.266617,-1.839277,-0.084049,0.584071,1.297509,...,-0.175430,-0.039216,0.992465,-0.621263,-0.824774,1.306676,-0.610195,-0.610195,-0.784796,-1.308923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2928458,-0.064673,1.593927,-0.353057,-2.454751,1.511309,0.522749,0.149793,-0.460784,0.216689,1.707596,...,-0.175430,-0.854002,-1.208534,1.234886,-0.824774,1.306676,-0.610195,-0.610195,1.230265,0.763987
4095447,-0.064673,-0.549907,0.236414,0.391922,-0.433329,0.368132,-0.479403,-0.084049,-3.875628,-0.471112,...,-0.175430,-1.785186,0.258799,-0.523571,1.445909,-0.925685,1.638819,1.638819,-0.784796,-1.308923
2105030,-0.064673,-0.549907,0.187180,-0.927457,-0.711134,0.522749,0.149793,-0.460784,0.216689,1.313057,...,-0.175430,0.243465,-1.208534,-0.621263,0.352617,0.190495,-0.610195,-0.610195,0.390656,0.763987
1601606,-0.064673,1.593927,-1.553287,0.143087,1.511309,0.522749,0.149793,-0.460784,0.216689,0.340740,...,-0.175430,-1.003657,0.992465,0.062581,0.331592,-1.011546,-0.610195,-0.610195,-0.113109,0.763987


In [154]:
Phat = xgb2.predict_proba(Testx)

In [185]:
Phat1 = model.predict_proba(Testx)

In [155]:
Phat

array([[0.7526304 , 0.2473696 ],
       [0.9210732 , 0.07892682],
       [0.89864445, 0.10135556],
       ...,
       [0.9146296 , 0.08537044],
       [0.97053677, 0.02946322],
       [0.92075616, 0.07924385]], dtype=float32)

In [186]:
Phat1

array([[0.7613046 , 0.2386954 ],
       [0.88726854, 0.11273148],
       [0.9545337 , 0.04546629],
       ...,
       [0.8402089 , 0.15979113],
       [0.94834113, 0.05165885],
       [0.885974  , 0.11402601]], dtype=float32)

In [187]:
Test_org['P(click)'] = Phat1[: , 1]

In [188]:
Test_org.head()

,id,P(click)
0,3295858251275419735,0.238695
1,12281702837842634283,0.112731
2,4638380339534007785,0.045466
3,17039804736879076347,0.100598
4,5753064066292192109,0.141601


In [190]:
Test_org = Test_org[["id", "P(click)"]]

Test_org.head()

,id,P(click)
0,3295858251275419735,0.238695
1,12281702837842634283,0.112731
2,4638380339534007785,0.045466
3,17039804736879076347,0.100598
4,5753064066292192109,0.141601


In [189]:
Test_org.to_csv('TestPhat2.csv',sep=',',na_rep="NA",header=True,index=False)

In [ ]:
##Thank you!!!